In [10]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# # Display the result
# cv2.imshow('Result', result)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [11]:
currentpath = os.getcwd()
def precision_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean(intersect/total_pixel_pred)
    return round(precision, 3)

def recall_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean(intersect/total_pixel_truth)
    return round(recall, 3)

def imshow(img):
    plt.imshow(img)
    plt.show()

def reportGen(info,img_path,data,data1):
    img_list =img_path[0]
    img_list=os.listdir(img_list)
    precision_p=np.sum(data)/len(img_list)
    recall_p=np.sum(data1)/len(img_list)
    html_string='<html><head> <style>table, th, td {border: 1px solid black;border-collapse: collapse;}</style><title></title></head><body>'
    html_string +="<h1>HardExudate Report</h1>"
    html_string +="<h2>Precision: "+  str(round(precision_p,3)) +"</h2>"
    html_string +="<h2>Recall: "+  str(round(recall_p,3)) +"</h2>"
    html_string +="<table><tr>"
    for head in range(0,len(info)):
        html_string +="<th width='40%'>"+ info[head] +"</th>"
    html_string +="</tr>"
    img_list.sort()
    for sl in range(0, len(img_list)):
        html_string +="<tr>"
        for path in range(0, len(img_path)):
            
            ld=os.listdir(img_path[path])
            html_string +="<td>"+ ld[sl] +"<br/><img width='350' src='"+ img_path[path] + ld[sl] +"'></td>"

        html_string +="<td>"+ str(data[sl]) +"</td>"
        html_string +="<td>"+ str(data1[sl]) +"</td>"
        html_string +="</tr>"

    html_string +='</table></body></html>'
    f = open('Results/_report.html','w')
    f.write(html_string)
    f.close()

In [12]:
image_path = "data/images/training/"
Precisions=[]
Recalls=[]
imgList=os.listdir(image_path)
results=[]
total_precision=0
for ins in range(1,len(imgList)+1):
    number="%02d" % (ins,)
    image_id="IDRiD_"+str(number)+".jpg"
    
    # Load the retinal image
    img = cv2.imread(image_path+image_id)


    # Image optic disk detection
    ye = cv2.addWeighted(R,0.73925,G,0.14675,0)
    ye = cv2.addWeighted(ye,1,B,0.114,0)
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(ye)
    cv2.circle(img,maxLoc,250,(0,0,0), -1)
    cv2.imwrite(os.path.join('Results/OpticalDisk/Training/'+image_id),img)
    
    # Image preprocessing reduce noise
    (R, G, B) = cv2.split(img) 
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))
    clahe_applied_image = clahe.apply(G)
    imageholder = cv2.fastNlMeansDenoising(clahe_applied_image,3,21,7)
    cv2.imwrite(os.path.join('Results/Prepos/Training/'+image_id),imageholder)


    # Threshold the image to create a binary mask
    _, tresh = cv2.threshold(img,np.percentile(img,65),255,cv2.THRESH_BINARY)

    # Create a structuring element for morphology operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))

    # Apply morphological operations to fill holes and smooth the image
    Opening = cv2.morphologyEx(tresh,cv2.MORPH_OPEN,kernel,iterations=1)
    Closing = cv2.morphologyEx(Opening,cv2.MORPH_CLOSE,kernel,iterations=1)

    # Find contours in the binary image
    zeros = np.zeros((2848,4288),dtype=np.uint8)
    # contours, hierarchy  = cv2.findContours(Closing,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    # largest_countour = sorted(contours,key=cv2.contourArea)
    # for cnt in range(0,len(largest_countour)):
    #     cv2.drawContours(zeros,[largest_countour[cnt]],-1,(255,255,255),-1)

    # Create a blank mask for the hard exudates
    mask = np.zeros_like(img[:,:,0])

    # Apply the mask to the original image
    results = cv2.bitwise_and(img, img, mask=mask)
    cv2.imwrite(os.path.join('Results/HardExodus/Training/'+image_id),mask)

    img_groundtruths=cv2.imread("data/groundtruths/training/hard exudates/IDRiD_"+str(number)+"_EX.tif",cv2.IMREAD_COLOR)
    precision = precision_score_(img_groundtruths,results)
    recall = recall_score_(img_groundtruths,results)
    Precisions.append(precision)
    Recalls.append(recall)

tablehead=[
    'Original Image',
    # 'OpticalDisk',
    'PreProcess',
    'ImageContour',
    'Hard Exudate',
    'Precision',
    'Recall']

original_img=os.path.join(currentpath,'data','images','Training')+'/'
# opticaldisk=os.path.join(currentpath,'Results','OpticalDisk','Training')+'/'
preprocess=os.path.join(currentpath,'Results','Prepos','Training')+'/'
contour=os.path.join(currentpath,'Results','Contour','Training')+'/'
hardexudate=os.path.join(currentpath,'Results','HardExodus','Training')+'/'
filepath=[original_img,preprocess, contour, hardexudate]
reportGen(tablehead,filepath,Precisions,Recalls)


/var/folders/zp/kn2c7zks0q55p6_3s3c4b97h0000gn/T/ipykernel_1294/3917330555.py:5: RuntimeWarning: invalid value encountered in scalar divide
  precision = np.mean(intersect/total_pixel_pred)


In [13]:
print(total_precision/len(imgList))

0.0
